In [ ]:
import networkx as nx
G = nx.karate_club_graph()  # NetworkX内置，空手道俱乐部
# G = nx.davis_southern_women_graph() #NetworkX内置，女性共现投影图
# G = nx.florentine_families_graph() #NetworkX内置，佛罗伦萨家族

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import torch

def plot_graph_from_adj(adj):
    """
    从任意邻接矩阵绘制无向无权图
    参数:
        adj: 邻接矩阵（支持NumPy数组或PyTorch张量）
    """
    # 1. 统一将邻接矩阵转为NumPy数组（处理PyTorch张量情况）
    if isinstance(adj, torch.Tensor):
        adj = adj.detach().cpu().numpy()  # 脱离计算图并转CPU（避免GPU tensor报错）
    
    # 2. 校验邻接矩阵合法性（必须是方阵）
    n = adj.shape[0]
    if adj.shape[1] != n:
        raise ValueError("邻接矩阵必须是方阵！")
    
    # 3. 从邻接矩阵重建无向图
    G = nx.Graph()
    G.add_nodes_from(range(n))  # 节点ID：0 ~ n-1
    
    # 遍历上三角（i<j），避免重复加边（无向图特性）
    for i in range(n):
        for j in range(i + 1, n):
            if adj[i, j] == 1:  # 仅保留有边的情况（无权图，1=有边，0=无边）
                G.add_edge(i, j)
    
    # 4. 绘制图形
    plt.figure(figsize=(8, 6))
    
    # 力导向布局（自动优化节点位置，适合任意图结构）
    pos = nx.spring_layout(G, seed=42)  # seed固定布局，每次运行结果一致
    
    # 绘制边（灰色半透明，避免杂乱）
    nx.draw_networkx_edges(G, pos, edge_color='#999999', alpha=0.6, width=1)
    # 绘制节点（浅蓝色带黑边框，清晰可见）
    nx.draw_networkx_nodes(G, pos, node_size=400, node_color='#87CEEB', edgecolors='black')
    # 绘制节点ID标签（黑色字体，大小适配）
    nx.draw_networkx_labels(G, pos, font_size=11, font_color='black')
    
    # 标题（显示节点数量）
    plt.title(f"Graph from Adjacency Matrix (Total Nodes: {n})", fontsize=12)
    plt.axis('off')  # 关闭坐标轴，聚焦图形
    plt.tight_layout()  # 自动调整布局，避免标签截断
    plt.show()

In [ ]:
A=torch.from_numpy(nx.to_numpy_array(G, weight=None)) 

In [ ]:
torch.sum(A)/2 #看一下边数

In [ ]:
plot_graph_from_adj(A)

In [ ]:
import networkx as nx
import itertools
from networkx.algorithms import isomorphism

class AdjacencySubgraphCounter:
    def __init__(self, graph=None):
        self.graph = graph if graph is not None else nx.Graph()
    
    def load_graph_from_edges(self, edges):
        """从边列表加载图"""
        self.graph.clear()
        self.graph.add_edges_from(edges)
    
    def load_graph_from_adjacency_matrix(self, adj_matrix):
        """从邻接矩阵加载图
        
        参数:
            adj_matrix: 二维列表或numpy数组，表示邻接矩阵
                        adj_matrix[i][j] = 1 表示节点i和j之间有边
                        adj_matrix[i][j] = 0 表示节点i和j之间无边
        """
        self.graph.clear()
        n = len(adj_matrix)
        
        # 检查是否为方阵
        for row in adj_matrix:
            if len(row) != n:
                raise ValueError("邻接矩阵必须是方阵")
        
        # 从邻接矩阵提取边（只处理上三角部分避免重复）
        edges = []
        for i in range(n):
            for j in range(i + 1, n):  # 无向图，i < j避免重复添加
                if adj_matrix[i][j] == 1:
                    edges.append((i, j))
        
        self.graph.add_edges_from(edges)
    
    def count_subgraphs(self, pattern):
        """计数与模式同构的所有子图（包括非诱导子图）"""
        pattern_nodes = len(pattern.nodes())
        graph_nodes = list(self.graph.nodes())
        counted = set()
        
        # 枚举所有节点组合（大小与模式一致）
        for node_comb in itertools.combinations(graph_nodes, pattern_nodes):
            # 找出这些节点在原图中存在的所有边
            possible_edges = []
            for u, v in itertools.combinations(node_comb, 2):
                if self.graph.has_edge(u, v):
                    possible_edges.append((u, v))
            
            # 生成这些边的所有可能子集（边数需与模式一致）
            pattern_edge_count = len(pattern.edges())
            for edge_subset in itertools.combinations(possible_edges, pattern_edge_count):
                # 构建当前子图
                subgraph = nx.Graph()
                subgraph.add_nodes_from(node_comb)
                subgraph.add_edges_from(edge_subset)
                
                # 检查节点数是否完整（避免边子集未连接所有节点）
                if len(subgraph.nodes()) != pattern_nodes:
                    continue
                
                # 检查同构
                gm = isomorphism.GraphMatcher(subgraph, pattern)
                if gm.is_isomorphic():
                    # 生成唯一标识
                    subgraph_id = (
                        frozenset(subgraph.nodes()),
                        frozenset(subgraph.edges())
                    )
                    if subgraph_id not in counted:
                        counted.add(subgraph_id)
        
        return len(counted)
    
    def create_pattern_from_edges(self, edges):
        """从边列表创建模式图"""
        pattern = nx.Graph()
        pattern.add_edges_from(edges)
        return pattern


# 初始化计数器并加载邻接矩阵
counter = AdjacencySubgraphCounter()
counter.load_graph_from_adjacency_matrix(A)


In [ ]:
import torch
def diag(matrix):
    """
    Input: An n×n PyTorch tensor (square matrix)
    Output: An n×n diagonal matrix with the same diagonal elements
    """
    # Extract the diagonal elements
    diag_elements = matrix.diagonal()
    
    # Create diagonal matrix using torch.diag()
    diag_matrix = torch.diag(diag_elements)
    
    return diag_matrix

In [ ]:
A2 = torch.matrix_power(A, 2)
A3 = torch.matrix_power(A, 3)
A4 = torch.matrix_power(A, 4)
A5 = torch.matrix_power(A, 5)
A6 = torch.matrix_power(A, 6)
A7 = torch.matrix_power(A, 7)

D2 = diag(A2)
D3 = diag(A3)
D4 = diag(A4)
D5 = diag(A5)
D6 = diag(A6)
D7 = diag(A7)

In [ ]:
import torch

def diag_matmul_left(D_diag, M):
    """
    高效计算 对角矩阵D × 普通矩阵M
    
    参数:
    D_diag (torch.Tensor): 对角矩阵D的对角元素，形状为 [n]
    M (torch.Tensor): 普通矩阵，形状为 [n, m]
    
    返回:
    torch.Tensor: 结果矩阵，形状为 [n, m]
    """
    diagonal = torch.diag(D_diag)           # 提取对角线元素 (n,)
    column_vector = diagonal.unsqueeze(1)   # 转换为列向量 (n, 1)
    return column_vector * M  # 等价于 D @ M，但更高效

def diag_matmul_right(M, D_diag):
    """
    高效计算 普通矩阵M × 对角矩阵D
    
    参数:
    M (torch.Tensor): 普通矩阵，形状为 [m, n]
    D_diag (torch.Tensor): 对角矩阵D的对角元素，形状为 [n]
    
    返回:
    torch.Tensor: 结果矩阵，形状为 [m, n]
    """
    return M * torch.diag(D_diag)  # 等价于 M @ D，但更高效

In [ ]:
AD2A = A@diag_matmul_left(D2,A)
A_A2= A*A2
D22 = D2*D2
A2D2 = diag_matmul_right(A2,D2)
D3A = diag_matmul_left(D3,A)
AD3 = diag_matmul_right(A,D3)
D2A  = diag_matmul_left(D2,A)
D2A2 = diag_matmul_left(D2,A2)
AD2 = diag_matmul_right(A,D2)
D2A3 = diag_matmul_left(D2,A3)
A2D2A = A2D2@A
dAD2Ad_A = diag_matmul_left(diag(AD2A),A)
AD3A = AD3@A
D3A2 = diag_matmul_left(D3,A2)
D4A = diag_matmul_left(D4,A)
AD4 = diag_matmul_right(A,D4)
bA_A2bA = (A*A2)@A
AD2A2 = AD2@A2
AbA_A2b = A@(A*A2)
D2D2A = diag_matmul_left((D2*D2),A)

In [ ]:
import torch

# 优化后的表达式
P6 = (A6 - (
    2*A5 + 2*diag_matmul_right(A,D2)@A3
    + A2D2@A2 + 2*AD3@A2 + diag_matmul_right(A,D4)@A + 2*diag_matmul_left(D3,A3)
    + 2*diag_matmul_right(A2,D4) + 2*diag_matmul_right(A,D5) + D6
) + (
    7*A4 + 2*diag_matmul_right(A,D22)@A
    + 7*A@diag_matmul_left(D2,A2) + 8*A@D3A + 7*A@(A2*A)@A
    + 10*(A*A2)@A2 + 14*D4@A 
    + 3*D5 + 6*(A*A2*A2)@A
    + 4*(D3*D3) + 6*A*A2*A3
    + A2*A2*A2
)-(
    18*AD2A + 19*A3 + 61*AD3 + 14*D4 + 31*A2*A2*A
)+(
    40*A2 + 52*D3
)- 12*A)


In [ ]:
torch.sum(P6)/2 #本文提出的P6计数公式运行结果
print(f"由本文提出的公式计算的6-路径数: {torch.sum(P6)/2}") 

In [ ]:
# 算法3计数6-路径（7节点6边）
path_pattern = counter.create_pattern_from_edges([(0,1), (1,2), (2,3), (3,4), (4,5), (5,6)])
print(f"由暴力算法计算的6-路径数: {counter.count_subgraphs(path_pattern)}") 